# ABM para Investigação de Dinâmicas Populacionais Urbanas

O objetivo de um ABM é buscar uma transição de fase definida por um comportamento emergente no espaço dos parâmetros do modelo.

O modelo original tinha 2 tipos de agente
- **Povo de Rua** (a) com um parâmetro de threshold que define uma regra de mudança;
- **Domicilios** (b) com um parâmetro de threshold que define uma regra de mudança;

Esses agentes ocupavam uma grade retangular MxN que continha apenas um atributo: a contagem de agentes do tipo (b) em cada célula. Para os agentes (a) a regra threshold forçava uma mudança (aleatória na grade) quando o número de agentes em sua célula do tipo (b) caia abaixo de um certo valor. Já os agentes do tipo (b) tinham uma regra de threshold que forçava uma mudança (aleatória na grade) quando o número de agente em sua célula crescia demais.

Os threshold eram definidos através de variáveis aleatórias, amostrada na criação de cada instância de um agente. O parâmetro deste modelo era justamente a relação entre Densidade de Agentes (tamanho da grade por número de instâncias de agentes), e os parâmetros das distribuições de threshold.

Falta um atributo de VALOR que seja composto por diversos outros atributos da grade;
Falta um agente de Segurança;
Falta atributos de Infraestrutura;

## FUNÇÃO VALOR
Consegue transformar uma célula da grade em valor para cada um dos agentes.

## AGENTE SEGURANÇA (c)
Tem um threshold sobre o valor(a * SA, b * SB) - se a soma deles cai para baixo de um threshold, ela se muda.
### Alteração do valor(a)
Deve dificultar a presença do agente tipo (a) - reduzindo o valor(a) de uma célular.

## FUNÇÃO INFRAESTUTURA
Uma função que atribui a cada célula um valor de infraestrutura.
### Alocação de Infraestrutura
Inicialmente faremos uma alocação usando ruído de Perlin. A cada etapa, o valor de N regiões (N sendo um parametro do modelo, chamado Orçamento) é incrementado. A função de ranqueamento tem que penalizar (temos?) celulas de alta infrasestrutura existente e beneficiar celulas de alta população.
#### SP Capital
A alocação de recursos é limitada pelos limites geopolíticos da cidade.
### Deterioração da Infraestrutura
A cada etapa da simulação, cada célula tem uma chance de perder N pontos de infraestrutura.

## AGENTE IGREJA (d)
Tem um threshold sobre o valor(a * IA, b * IB) - se a soma deles cai para baixo de um threshold, ela se muda.
### Alteração do valor(a)
A Igreja busca construir um equilibrio da população do agente (a) - ela altera o valor da celula para agentes (a) nos dois sentidos: se o valor está abaixo de um threshold, ela o aumenta, se ele esta acima, ela o reduz.

# Framework de Desenvolvimento

- Começar implantando infraestrutura.
- Adicionar agentes (c) e (d)
 

In [23]:
import random

class Grid:
    '''
    '''
    def __init__(self, x, y):
        '''

            loc_dict Dict: dicionário para conter atributos do modelo e agentes.
            loc_pairs List: lista com pares de coordenadas (usada para amostragem aleatória)
        '''
        self.x_size = x
        self.y_size = y
        self.agent_set = set()
        self.loc_dict = dict()
        self.loc_pairs = [(i, j) for i in range(x) for j in range(y)]
        for i in range(x):
            for j in range(y):
                self.loc_dict[(i,j)] = dict()
                self.loc_dict[(i,j)]['infrastructure'] = 0
                self.loc_dict[(i,j)]['agent_present'] = set()

    def return_coord_pair(self):
        '''
        Retorna um par de coordenadas aleatoriamente.
        '''
        return random.choice(self.loc_pairs)

    def add_agent_position(self, pos_tuple, agent):
        '''
        Adiciona um agente na posição x, y = pos_tuple.
        '''
        self.loc_dict[(pos_tuple[0], pos_tuple[1])]['agent_present'].add(agent)

    def change_agent_position(self, agent, new_pos):
        '''
        '''
        self.loc_dict[(agent.x_pos, agent.y_pos)]['agent_present'].remove(agent)
        self.loc_dict[(new_pos[0], new_pos[1])]['agent_present'].add(agent)

class Agent:
    '''
    Classe representando um Agente genérico

    '''
    def __init__(self, grid):
        '''
        Parameters:
            grid Grid: objeto Grid onde o agente residirá
        Attributes:
            pos Tuple: tuple com posição x, y do agente no Grid
        '''
        self.x_pos, self.y_pos = grid.return_coord_pair()
        self.grid = grid
        self.grid.add_agent_position((self.x_pos, self.y_pos), self)

    def move(self):
        new_pos = self.grid.return_coord_pair()
        self.grid.change_agent_position(self, new_pos)
        self.x_pos, self.y_pos = new_pos

class AgentA(Agent):
    '''
    Classe representando o Povo de Rua
    '''
    def __init__(self, grid, thresh_dict):
        super().__init__(grid)
        self.thresh_dict = thresh_dict

    def regra_movimento(self):
        '''
        Avalia atributos da Grid para determinar se agente se muda ou não
        '''
        condicao = True
        for threshold in self.thresh_dict.keys():
            if self.thresh_dict[threshold] < self.grid.loc_dict[(self.x_pos, self.y_pos)][threshold]:
                condicao = True
                break
        if condicao:
            self.move()

In [24]:
teste = Grid(10, 10)
a = AgentA(teste)